In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
from tqdm import tqdm

In [2]:
class card:
    """
    Defines structure for a card
    
    """
    def __init__(self,color,suit,number,speciality,face):
        """
        Args:
            color ==> red or black
            suit ===> club, heart, diamond, spade
            number ==> 1 to 13
            speciality ==> None or Ace, King, Queen,Jack
        """
        self.suit = suit
        self.color = color
        self.number = number
        self.speciality = speciality
        self.face = face

In [3]:
class state:
    """
    Defines the structure of a state. A state holds pile, foundation and tableau, cards to review. 
    
    """
    def __init__(self):
        
        """
        A state comprises of :
            4 foundations ==> club, heart, diamond, spade in that order
        
            7 tableaus ===> cards facing up or down
            
            1 pile ===> empty or has cards
        
        """
        
        self.all_cards = []
        self.pile = None
        self.tableau = [[] for i in range(7)]
        self.foundation = [[],[],[],[]]
        
        self.gen_non_special_cards('red','heart')
        self.gen_non_special_cards('red','diamond')
        self.gen_non_special_cards('black','spade')
        self.gen_non_special_cards('black','club')
        
        
        self.gen_special_cards('red','heart')
        self.gen_special_cards('red','diamond')
        self.gen_special_cards('black','spade')
        self.gen_special_cards('black','club')
        
        #print(len(self.all_cards))
        
        random.shuffle(self.all_cards)
        
        pile_index = self.make_pile()
        _ = self.make_tableau(pile_index)
        
        self.hashable_state = None
    def make_pile(self):
        
        pile_index = random.sample(range(len(self.all_cards)),24 )
        
        pile_index.sort()
        self.pile = [self.all_cards[i] for i in pile_index]
        
        for card in self.pile:
            card.face = 'up'
        #print(self.pile)
        
        return pile_index
    
   
            
    def make_tableau(self,pile_index):
        
        #print(pile_index)
        tableau_index = [i for i in range(len(self.all_cards)) if i not in pile_index]
        check1 = tableau_index[:]
        check2 = []
        #print("index of cards to be in tableau ", tableau_index)
        
        for i in range(1,8):
            
            all_cards_this_tableau_index = random.sample(range(len(tableau_index)),i)
            
            all_cards_this_tableau = [tableau_index[x] for x in all_cards_this_tableau_index]
            #print("index of cards to be in {} tableau".format(i),all_cards_this_tableau)
            
            tableau_index = [x for x in tableau_index if x not in all_cards_this_tableau]
            
            for card_index  in all_cards_this_tableau:
                
                
                self.tableau[i-1].append(self.all_cards[card_index])
                check2.append(card_index)
        
        
        
        for i in range(7):
            self.tableau[i][-1].face = 'up'
            
        check1.sort()
        check2.sort()
        
        #print(check1)
        #print(check2)
        
        assert check1 == check2
        #print(a)
                
    def gen_special_cards(self,color,suit,face = 'down'):
        
        self.all_cards.append(card(color,suit,1,'ACE',face))
        self.all_cards.append(card(color,suit,13,'KING',face))
        self.all_cards.append(card(color,suit,12,'QUEEN',face))
        self.all_cards.append(card(color,suit,11,'JACK',face))
        
        
    def gen_non_special_cards(self,color,suit,face = "down",speciality = None):
        
        for number in range(2,11):
            self.all_cards.append(card(color,suit,number,speciality,face))

In [4]:

class env:
    
    def __init__(self):
        
        self.state = state()
        self.action_n = 6
        self.hashable_state = None
        #print("called")
        
        self.hashable_map = {}
        self.map = {}
        self.number_of_states = 0
        
    def reset(self):
        self.state = state()
    
    def generate_deep_copy_state(self,to_copy_state):
        
        deep_copy_state = state()
        
        deep_copy_state.pile = []
        deep_copy_state.tableau = [[] for _ in range(7)]
        deep_copy_state.foundation = [[] for _ in range(4)]
        
        for cd in to_copy_state.pile:
            new_card = card(cd.color,cd.suit,cd.number,cd.speciality,cd.face)
            deep_copy_state.pile.append(new_card)
            
            
        for i in range(7):
            
            for cd in to_copy_state.tableau[i]:
                new_card = card(cd.color,cd.suit,cd.number,cd.speciality,cd.face)

                deep_copy_state.tableau[i].append(new_card)
                
                
                
                
        for i  in range(4):
            for cd in to_copy_state.foundation[i]:
                new_card = card(cd.color,cd.suit,cd.number,cd.speciality,cd.face)

                deep_copy_state.foundation[i].append(new_card)
                
                
        
        return deep_copy_state
        
    def current_state(self):
        return self.state
    
    def generate_hashable_state(self,state):
        
        hashable_state = []
        
        #for card in self.state.pile:
            
        hashable_state.append(tuple(state.pile))
            
        
        for i in range(7):
            
            
            hashable_state.append(tuple(state.tableau[i]))
                
                
        
        for i in range(4):
            
            hashable_state.append(tuple(state.foundation[i]))
            
            
         
        
        return tuple(hashable_state)
        
    def step(self,action):
        
        taken = None
        if action == 0 :
            taken = self.tableau_to_foundation_reveal()
            
        elif action == 1:
            taken = self.to_foundation_stack()
            
        elif action == 2:
            taken = self.tableau_to_tableau_reveal()
            
        elif action == 3:
            taken = self.pile_to_tableau()
            
        elif action == 4:
            taken = self.foundation_to_tableau()
            
        elif action == 5:
            taken = self.tableau_to_tableau_not_reveal()
    
        
        key = self.generate_hashable_state(self.state)
        
        #print(tuple(self.hashable_state))
        
        ##key = tuple(self.hashable_state)
        if key not in self.map:
            self.map[key] = 0
            
        self.map[key]+=1
        
        
        return self.isterminal(),taken
    
    
    
    def print_card(self,card):
        print("suit = {:10s} color = {:10s} number = {:5} speciality = {:10s} face = {:5s}".format(card.suit,card.color,card.number,str(card.speciality),card.face))

    def print_cards(self,cards):
        
        for card in cards:
            self.print_card(card)
    
    def check_compatible(self,card):
        
        f = self.suit_number(card.suit)
        
        if card.number == 1 or (len(self.state.foundation[f])> 0 and card.number == self.state.foundation[f][-1].number+1):
            return True
                
                
        return False
                
        
    def suit_number(self,suit):
        
        if suit == 'club':
            return 0
        elif suit == 'heart':
            return 1 
        elif suit == 'diamond':
            return 2
        else:
            return 3
    def highlight_movable_cards_pile(self):
        
        movable_where = []
        
        movable_indices_tableau = []
        movable_indices_foundation = []
        for i,card in enumerate(self.state.pile):
            
            movable_where.append([])
            
            movable_where[i] = {'tableau':[],'foundation':[]}
            
            for f in range(7):
                
                cond1 = len(self.state.tableau[f]) == 0 and  card.number == 13
                
                cond2 = len(self.state.tableau[f]) > 0 and self.state.tableau[f][-1].color != card.color and card.number+1 == self.state.tableau[f][-1].number
                
                if cond1 or cond2:
                    movable_where[i]['tableau'].append(f)
                    
            
            if self.check_compatible(card):
                movable_where[i]['foundation'].append(self.suit_number(card.suit))
             
            
            if len(movable_where[i]['tableau']) !=0 :
                movable_indices_tableau.append(i)
                
            if len(movable_where[i]['foundation']) != 0:
                movable_indices_foundation.append(i)
                
        return movable_where,movable_indices_tableau,movable_indices_foundation
            
        
    def tableau_to_foundation_reveal(self):
        
        movable = []
        moves = []
        for i in range(7):
            
            movable.append([])
            
            if len(self.state.tableau[i]) <= 1 or self.state.tableau[i][-2].face == 'up': 
                continue
            
            to_move_card = self.state.tableau[i][-1]
            
            f_no = self.suit_number(to_move_card.suit)
            
            if to_move_card.number == 1:
                moves.append(i)
                
            elif len(self.state.foundation[f_no])>0 and to_move_card.number == self.state.foundation[f_no][-1].number +1:
                moves.append(i)
            
            
        
        
        if len(moves) == 0:
            return False
        
        
        
        
        mp = {}
        len_keys = 0
        len_moves = len(moves)
        while len_keys<len_moves:
            deep_copy_state = self.generate_deep_copy_state(self.state)
            ind = None
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp:
                    mp[ind] = 1
                    len_keys+=1
                    break
            from_tableau = moves[ind]

            #print("tableau number ", from_tableau)

            card = deep_copy_state.tableau[from_tableau][-1]

            deep_copy_state.tableau[from_tableau].pop()

            deep_copy_state.tableau[from_tableau][-1].face = 'up'

            deep_copy_state.foundation[self.suit_number(card.suit)].append(card)
            
            hashable_state = self.generate_hashable_state(deep_copy_state)
            
            if hashable_state not in self.hashable_map:
                
                self.state = deep_copy_state
                
                self.hashable_map[hashable_state] = 1
                
                return True
        
        return False
    def to_foundation_stack(self):
        
        
        moves = []
        
        
        for i,card in enumerate(self.state.pile):
            
            f_no = self.suit_number(card.suit)
            
            if card.number == 1 or (len(self.state.foundation[f_no]) > 0 and card.number == self.state.foundation[f_no][-1].number + 1):
                moves.append((0,i))
                
            
            
        
        for i in range(7):
            
            if len(self.state.tableau[i]) == 0:
                continue
                
            card = self.state.tableau[i][-1]
            
            f_no = self.suit_number(card.suit)
            
            if card.number == 1 or (len(self.state.foundation[f_no]) > 0 and card.number == self.state.foundation[f_no][-1].number + 1):
                moves.append((1,i))
                
                
          
        if len(moves) == 0:
            return False
        
        
        
        mp = {}
        len_keys = 0
        len_moves = len(moves)
        
        
        while len_keys <len_moves:
            
            
            deep_copy_state = self.generate_deep_copy_state(self.state)
            ind = None
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp:
                    mp[ind] = 1
                    len_keys+=1
                    break
                    
                    
            #ind = random.sample(range(len(moves)),1)[0]
            
            
            
            typ, i = moves[ind]

            if typ == 0:

                card = deep_copy_state.pile[i]
                f_no = self.suit_number(card.suit)

                deep_copy_state.pile.pop(i)


                deep_copy_state.foundation[f_no].append(card)
            else:

                card = deep_copy_state.tableau[i][-1]

                f_no = self.suit_number(card.suit)

                deep_copy_state.tableau[i].pop()

                if len(deep_copy_state.tableau[i]) >0:

                    deep_copy_state.tableau[i][-1].face = 'up'

                deep_copy_state.foundation[f_no].append(card)

            
            hashable_state = self.generate_hashable_state(deep_copy_state)
            
            if hashable_state not in self.hashable_map:
                
                self.state = deep_copy_state
                
                self.hashable_map[hashable_state] = 1
                
                return True
            #return True
        
        return False
    def tableau_to_tableau_reveal(self):
        
        movable = []
        
        for i in range(7):
            
            movable.append([])
            if len(self.state.tableau[i])<=1:
                continue
            
            prev = None
            for j,card in enumerate(self.state.tableau[i]):
                
                if card.face == 'up' and prev is not None and prev.face == 'down':
                    movable[i].append(j)
                    break
                    
                prev = card 
                
         
        #print("In tableu to tableu",movable)
        
        moves = []
        
        for i in range(7):
            
            for j in range(7):
                
                if i == j or len(movable[i])==0 or len(self.state.tableau[j])==0:
                    continue
                   
                
                to_move_card = self.state.tableau[i][movable[i][0]]
                
                last_card = self.state.tableau[j][-1]
                    
                
                cond1 = to_move_card.color != last_card.color
                
                cond2 = to_move_card.number + 1 == last_card.number
                
                if cond1 and cond2:
                    
                    moves.append((i,movable[i][0],j))
                    
                    
        
        if len(moves)==0:
            return False
        
        
        
        mp = {}
        len_keys = 0
        len_moves = len(moves)
        
        while len_keys < len_moves:
            
            deep_copy_state = self.generate_deep_copy_state(self.state)
            ind = None
            
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp:
                    mp[ind] = 1
                    len_keys+=1
                    break
        
            #ind = random.sample(range(len(moves)),1)[0]
           
            from_tableau,i,to_tableau = moves[ind]

            for card in deep_copy_state.tableau[from_tableau][i:]:

                deep_copy_state.tableau[to_tableau].append(card)



            deep_copy_state.tableau[from_tableau] = deep_copy_state.tableau[from_tableau][:i]

            deep_copy_state.tableau[from_tableau][-1].face = 'up'
            
            
            hashable_state = self.generate_hashable_state(deep_copy_state)
            
            if hashable_state not in self.hashable_map:
                
                self.state = deep_copy_state
                
                self.hashable_map[hashable_state] = 1
                
                return True

            #return True
        
        return False
    def pile_to_tableau(self):
        
        movable_where,movable_indices_tableau,movable_indices_foundation  = self.highlight_movable_cards_pile()
        
        if len(movable_indices_tableau) == 0:
            return False
        
        
        
        mp1,mp2 = {},{}
        len_keys = 0
        len_moves = len(movable_indices_tableau)
        
        while len_keys < len_moves:
            
            deep_copy_state = self.generate_deep_copy_state(self.state)
            ind = None
            
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp1:
                    mp1[ind] = 1
                    len_keys+=1
                    break
        
            #ind = random.sample(range(len(movable_indices_tableau)),1)[0]



            to_move = movable_indices_tableau[ind]

            card_to_move = deep_copy_state.pile[to_move]

            
            
            tableau_index = random.sample(range(len(movable_where[to_move]['tableau'])),1)[0]

            tableau_index = movable_where[to_move]['tableau'][tableau_index]

            deep_copy_state.tableau[tableau_index].append(card_to_move)




            deep_copy_state.pile.pop(to_move)
        
            hashable_state = self.generate_hashable_state(deep_copy_state)
            
            if hashable_state not in self.hashable_map:
                
                self.state = deep_copy_state
                
                self.hashable_map[hashable_state] = 1
                
                return True
            
        return True
        
    def foundation_to_tableau(self):
        
        moves = []
        for i in range(4):
            
            if len(self.state.foundation[i]) == 0:
                continue
                
            card = self.state.foundation[i][-1]
            
            for j in range(7):
                
                if len(self.state.tableau[j]) == 0:
                    continue
                    
                
                last_card = self.state.tableau[j][-1]
                
                if last_card.color != card.color and last_card.number == card.number+1:
                    moves.append((i,j))
        
        
        
        if len(moves) == 0:
            return False
        
        mp = {}
        len_keys = 0
        len_moves = len(moves)
        
        while len_keys < len_moves:
            
            deep_copy_state = self.generate_deep_copy_state(self.state)
            #print(len(self.state.foundation[foundation]),len(deep_copy_state.foundation[foundation]))

            ind = None
            
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp:
                    mp[ind] = 1
                    len_keys+=1
                    break
                    
            #ind = random.sample(range(len(moves)),1)[0]

            foundation,tableau = moves[ind]
            
            #print(foundation,tableau)
            #print(len(self.state.foundation[foundation]),len(deep_copy_state.foundation[foundation]))
            card = deep_copy_state.foundation[foundation][-1]

            deep_copy_state.foundation[foundation].pop()

            deep_copy_state.tableau[tableau].append(card)



            hashable_state = self.generate_hashable_state(deep_copy_state)

            if hashable_state not in self.hashable_map:

                self.state = deep_copy_state

                self.hashable_map[hashable_state] = 1

                return True
        
        return False
    def tableau_to_tableau_not_reveal(self):
        
        
        movable = []
        
        for i in range(7):
            
            first = False
            movable.append([])
            for j,card in enumerate(self.state.tableau[i]):
                
                if card.face == 'up' and i == 0:
                    
                    movable[i].extend(range(len(self.state.tableau[i])))
                    
                    break
                elif card.face == 'up' and first == True:
                    
                    movable[i].append(j)
                
                elif card.face == 'up':
                    first = True
                    
        #to_move = []
        
        where_to_move = []
        
        which_movable_tableaus = []
        
        
        for i in range(7):
            
            #to_move.append([])
            where_to_move.append([])
            for j,to_move in enumerate(movable[i]):
            
                for k in range(7):

                    if i == k:
                        continue

                        to_move_card = self.state.tableau[i][to_move]

                        

                        condition1 = len(self.state.tableau[k])>0 and to_move_card.color != self.state.tableau[k][-1].color  
                        condition2 = len(self.state.tableau[k])>0 and to_move_card.number+1 == self.state.tableau[k][-1].number
                        
                        if condition1 and condition2:
                            where_to_move[i].append(j)
                            
                            which_movable_tableaus.append((i,j,k))
                        else:
                            
                            where_to_move[i].append(np.inf)
                            
                 
        
        if len(which_movable_tableaus) == 0:
            return False
        
        mp = {}
        len_keys = 0
        len_moves = len(moves)
        
        while len_keys < len_moves:
            
            deep_copy_state = self.generate_deep_copy_state(self.state)
            ind = None
            
            while True:
                ind = random.sample(range(len_moves),1)[0]
                if ind not in mp:
                    mp[ind] = 1
                    len_keys+=1
                    break
                    
                    
            #ind = random.sample(range(len(which_movable_tableaus)),1)[0]

            from_tableau, cards_to_move,to_tableau = which_movable_tableaus[ind]





            for card in deep_copy_state.tableau[from_tableau[cards_to_move:]]:

                deep_copy_state.tableau[to_tableau].append(card)


            deep_copy_state.tableau[from_tableau] = deep_copy_state.tableau[from_tableau[:cards_to_move]]


            hashable_state = self.generate_hashable_state(deep_copy_state)

            if hashable_state not in self.hashable_map:

                self.state = deep_copy_state

                self.hashable_map[hashable_state] = 1

                return True
        
        return False
        
    def get_pile(self):
        
        self.print_cards(self.state.pile)
        
    def get_tableau(self):
        
        for i,tableau in enumerate(self.state.tableau):
            print("printing tableau number {} ********************".format(i+1))
            self.print_cards(tableau)
            
    def get_foundation(self):
        
        for i,foundation in enumerate(self.state.foundation):
            print("printing foundation number {} ********************".format(i+1))
            self.print_cards(foundation)
        
            
    def isterminal(self):
        
        ans = (len(self.state.pile) == 0)
        
        #print(ans)
        for i in range(len(self.state.tableau)):
            ans = ans and (len(self.state.tableau[i]) == 0 )
            
            
        #print(ans)
        
        for i in range(len(self.state.foundation)):
            ans = ans and (len(self.state.foundation[i])==13)
            
            
        #print(ans)   
        return ans   
        
    

In [5]:
#en = env()

In [6]:
#en.get_pile()

In [7]:
#en.get_tableau()

In [8]:
#en.get_foundation()

In [9]:
#en.step(1)

In [10]:
import time
start = time.time()


count = 0
actions = [0]*6
fail = [0]*6
different_states = []
for j in tqdm(range(1000)):
    en = env()
    for i in range(10000):

        action = random.sample(range(6),1)[0]
        
        actions[action]+=1

        won,failed = en.step(action)
        
        if failed:
            fail[action]+=1
        #print(i+1,action,won)
        if won == True:
            count+=1
            break
    different_states.append(len(en.hashable_map.keys()))

print(count)
print(time.time()-start)

100%|██████████| 1000/1000 [16:42<00:00,  1.01it/s]

9
1002.5705687999725


In [11]:
len(en.hashable_map.keys()),min(different_states),max(different_states)

(2619, 10, 3375)

In [17]:
import pandas as pd 
pd.Series(different_states).value_counts(),max(different_states)

(8901     57
 10916    54
 8899     49
 10919    43
 12314    41
          ..
 53        1
 55        1
 59        1
 61        1
 16285     1
 Length: 761, dtype: int64, 444456)

In [97]:
s = 0
for i in range(12):
    
    print(len(en.hashable_state[i]))
    s+= len(en.hashable_state[i])
    
print(s)

15
1
3
7
4
4
8
7
1
0
2
0
52


In [15]:
fail

[112334, 5164648, 620370, 1303271, 4617087, 0]

In [91]:
en.get_pile()

suit = diamond    color = red        number =    10 speciality = None       face = up   
suit = spade      color = black      number =     4 speciality = None       face = up   
suit = spade      color = black      number =     8 speciality = None       face = up   
suit = club       color = black      number =     4 speciality = None       face = up   
suit = club       color = black      number =    10 speciality = None       face = up   
suit = heart      color = red        number =    12 speciality = QUEEN      face = up   
suit = diamond    color = red        number =    13 speciality = KING       face = up   
suit = diamond    color = red        number =    12 speciality = QUEEN      face = up   
suit = club       color = black      number =    13 speciality = KING       face = up   


In [92]:
en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = up   
printing foundation number 2 ********************
suit = heart      color = red        number =     1 speciality = ACE        face = up   
printing foundation number 3 ********************
suit = diamond    color = red        number =     1 speciality = ACE        face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =     3 speciality = None       face = up   
printing foundation number 4 ********************
suit = spade      color = black      number =     1 speciality = ACE        face = up   
suit = spade      color = black      number =     2 speciality = None       face = up   


In [93]:
en.get_tableau()

printing tableau number 1 ********************
suit = heart      color = red        number =     7 speciality = None       face = up   
suit = spade      color = black      number =     6 speciality = None       face = up   
printing tableau number 2 ********************
suit = heart      color = red        number =    13 speciality = KING       face = up   
suit = club       color = black      number =    12 speciality = QUEEN      face = up   
printing tableau number 3 ********************
suit = spade      color = black      number =     3 speciality = None       face = down 
suit = heart      color = red        number =    10 speciality = None       face = up   
suit = club       color = black      number =     9 speciality = None       face = up   
suit = heart      color = red        number =     8 speciality = None       face = up   
suit = spade      color = black      number =     7 speciality = None       face = up   
suit = heart      color = red        number =     6 specia

In [ ]:
import time
start = time.time()


count = 0
actions = [0]*6
fail = [0]*6

number_of_games = 0

for number_of_games in tqdm(range(10000)):
    
    
    en = env()
    tot = 20000

    won,taken = False,True
    number_of_actions = 0
    while number_of_actions < tot and won == False:

        action = 0
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            number_of_actions+=1

        action +=1
        taken = True
        
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            number_of_actions+=1

        action +=1
        taken = True
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            
            number_of_actions+=1

        action +=1
        taken = True
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            number_of_actions+=1

        action +=1
        taken = True
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            number_of_actions+=1



        action +=1
        taken = True
        while taken == True and number_of_actions < tot and won == False:
            won,taken = en.step(action)
            number_of_actions+=1


        

        if won:
            count+=1
         
    different_states.append(len(en.hashable_map.keys()))
    if number_of_games%100 == 0:
        print(count,number_of_games)

print(count/10)
print(time.time()-start)

  0%|          | 1/10000 [00:00<1:23:31,  2.00it/s]

0 0


  1%|          | 101/10000 [05:35<7:27:31,  2.71s/it]

1 100


  2%|▏         | 201/10000 [11:19<10:49:34,  3.98s/it]

...

 19%|█▉        | 1901/10000 [1:45:01<6:45:54,  3.01s/it] 

9 1900


 20%|██        | 2001/10000 [1:50:09<5:48:23,  2.61s/it] 

11 2000


 20%|██        | 2017/10000 [1:51:02<8:32:27,  3.85s/it]

In [19]:
pd.DataFrame(data={'value':different_states}).value.value_counts()

8901     57
10916    54
8899     49
10919    43
12314    41
         ..
53        1
55        1
59        1
61        1
16285     1
Name: value, Length: 761, dtype: int64

In [20]:
max(different_states)

444456

In [ ]:
en.get_foundation()

In [ ]:
en.get_tableau()

In [53]:
en.get_pile()

suit = heart      color = red        number =     2 speciality = None       face = up   
suit = heart      color = red        number =     3 speciality = None       face = up   
suit = heart      color = red        number =     4 speciality = None       face = up   
suit = heart      color = red        number =     6 speciality = None       face = up   
suit = heart      color = red        number =     7 speciality = None       face = up   
suit = heart      color = red        number =     9 speciality = None       face = up   
suit = diamond    color = red        number =     2 speciality = None       face = up   
suit = diamond    color = red        number =     3 speciality = None       face = up   
suit = diamond    color = red        number =     7 speciality = None       face = up   
suit = diamond    color = red        number =    10 speciality = None       face = up   
suit = spade      color = black      number =     3 speciality = None       face = up   
suit = spade      col

In [76]:
tuple([])

()

In [48]:
easy_game_1 = None

easy_game_1 = state()

suit = {0:'club',1:'heart',2:'diamond',3:'spade'}

speciality = {1:'ACE',11:'KING',12:'QUEEN',13:'JACK'}

color = {'club':'black','spade':'black','diamond':'red','heart':'red'}
for i in range(2,11):
    speciality[i]=None

for j in range(4):
    for i in range(1,14):
        
        easy_game_1.foundation[j].append(card(color[suit[j]],suit[j],i,speciality[i],'face-up'))

        
for i in range(7):
    easy_game_1.tableau[i] = []
    
easy_game_1.pile = []

for i in range(4):
    easy_game_1.pile.append(easy_game_1.foundation[i].pop())
    

In [49]:
easy_en = env()

In [50]:
easy_en.state = easy_game_1

In [51]:
easy_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = face-up
suit = club       color = black      number =     2 speciality = None       face = face-up
suit = club       color = black      number =     3 speciality = None       face = face-up
suit = club       color = black      number =     4 speciality = None       face = face-up
suit = club       color = black      number =     5 speciality = None       face = face-up
suit = club       color = black      number =     6 speciality = None       face = face-up
suit = club       color = black      number =     7 speciality = None       face = face-up
suit = club       color = black      number =     8 speciality = None       face = face-up
suit = club       color = black      number =     9 speciality = None       face = face-up
suit = club       color = black      number =    10 speciality = None       face = face-up
suit = club       color = black      num

In [52]:
easy_en.get_pile()

suit = club       color = black      number =    13 speciality = JACK       face = face-up
suit = heart      color = red        number =    13 speciality = JACK       face = face-up
suit = diamond    color = red        number =    13 speciality = JACK       face = face-up
suit = spade      color = black      number =    13 speciality = JACK       face = face-up


In [53]:

action_type = {0:'tableau_to_foundation_reveal',\
               1:'to_foundation_stack',2:'tableau_to_tableau_reveal',\
               3:'pile_to_tableau',\
                4:'foundation_to_tableau',\
               5:'tableau_to_tableau_not_reveal'\
              }

action_freq = {0:0,1:0,2:0,3:0,4:0,5:0}


for steps in range(100):
    
    action = random.sample(range(6),1)[0]
       
    action_freq[action]+=1

    won,failed = easy_en.step(action)
        
      
    if won == True:
        break
        
print(won)
print(action_freq)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
False
False
True
True
True
True
{0: 5, 1: 4, 2: 2, 3: 3, 4: 7, 5: 3}


In [54]:
easy_en.get_pile()

In [44]:
easy_en.get_foundation()

printing foundation number 1 ********************
suit = club       color = black      number =     1 speciality = ACE        face = face-up
suit = club       color = black      number =     2 speciality = None       face = face-up
suit = club       color = black      number =     3 speciality = None       face = face-up
suit = club       color = black      number =     4 speciality = None       face = face-up
suit = club       color = black      number =     5 speciality = None       face = face-up
suit = club       color = black      number =     6 speciality = None       face = face-up
suit = club       color = black      number =     7 speciality = None       face = face-up
suit = club       color = black      number =     8 speciality = None       face = face-up
suit = club       color = black      number =     9 speciality = None       face = face-up
suit = club       color = black      number =    10 speciality = None       face = face-up
suit = club       color = black      num

In [45]:
easy_en.get_tableau()

printing tableau number 1 ********************
printing tableau number 2 ********************
printing tableau number 3 ********************
printing tableau number 4 ********************
printing tableau number 5 ********************
printing tableau number 6 ********************
printing tableau number 7 ********************


In [46]:
len(easy_en.state.pile)

0

In [47]:
easy_en.isterminal()

False